In [19]:
import torch

import torch.nn.functional as F
from core.tokenizers.regex import get_tokenizer
from core.utils import get_param_count
from core.config import device
from core.models import Generator

In [8]:
tokenizer = get_tokenizer("poems.txt", 1024, "tokenizer/en", True)

In [22]:
BLOCK_SIZE = 32
BATCH_SIZE = 64
EMBEDDING_SIZE = 256
TRAIN_FILE = "datasets/train.txt"
TEST_FILE = "datasets/test.txt"
train_data = tokenizer.encode(open(TRAIN_FILE, encoding="utf-8").read())
test_data = tokenizer.encode(open(TEST_FILE, encoding="utf-8").read())

In [28]:
def get_batch(split):
    data = train_data if split == "train" else test_data
    idxs = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, ))
    X = torch.stack([torch.tensor(data[idx: idx + BLOCK_SIZE], device=device) for idx in idxs])
    Y = torch.stack([torch.tensor(data[idx + 1: idx + 1 + BLOCK_SIZE], device=device) for idx in idxs])
    return X, Y

In [36]:
x, y = get_batch("train")
print(x[0].tolist())
print(y[0].tolist())

[115, 511, 44, 431, 470, 115, 444, 406, 101, 418, 33, 10, 317, 261, 357, 647, 116, 116, 321, 386, 44, 432, 101, 103, 103, 292, 44, 470, 367, 44, 288, 258]
[511, 44, 431, 470, 115, 444, 406, 101, 418, 33, 10, 317, 261, 357, 647, 116, 116, 321, 386, 44, 432, 101, 103, 103, 292, 44, 470, 367, 44, 288, 258, 39]
